# core

> Base classes and routes for authentication, and dealing with api responses.

- `DomoAuth` encapsulates authentication.  Can init with username/password or access_token
- For consistency, use `domo_api_request` to handle API requests in route functions.  `domo_api_requests` and `route functions` should always return an instance of `ResponseGetData` class.  For streaming responses, use `domo_api_stream_request` to automagically download API responses in route functions.
- `Route Functions` should always throw an error if not `ResponseGetData.is_success`

Features or classes are implemented as separate notebooks and then exported into their own module folder (see `default_exp` line in each .ipynb file)
- each feature should be implemented as series of route functions ('GET', 'PUT', 'POST' requests), then a `@dataclass` class may be used to represent the feature with methods that wrap and provide logic around route functions.
    - ex. an UPSERT method might wrap logic to test if the entity already exists (in which case call the PUT route) or create the entity if it doesn't already exist ('POST')
- `@classmethods` return an instance of the class
    - ex. each class should have a `get_by_id` method which returns an instance of the entity (typically the result of the search datacenter API)

Implementations, take a set of features and create a use case or project
    - ex.back up code engine and custom app (features) into appdb collections (feature)

In [1]:
# | default_exp client.core

In [2]:
# | exporti
from dataclasses import dataclass, field
from typing import Callable

import json
import requests
import time

from pprint import pprint

import mbison.utils as dmut

In [3]:
# | hide

import os

In [4]:
# | export


def get_jupyter_account(account_name, domojupyter_fn: Callable):
    creds_json = None
    i = 0

    while not creds_json:
        try:
            creds_json = json.loads(
                domojupyter_fn.get_account_property_value(account_name, "credentials")
            )

        except Exception as e:
            print(f"account error - retrying {e}")
            i = i + 1
            if i > 12:
                return
            time.sleep(5)

    return creds_json

## DomoAuth class


In [12]:
# | export


@dataclass
class DomoAuth:
    domo_instance: str

    username: str = None
    password: str = field(repr=False, default=None)
    access_token: str = field(repr=False, default=None)
    session_token: str = field(repr=False, default=None)

    @classmethod
    def from_creds_json(
        cls,
        creds_json: dict,
    ):
        username = creds_json.get("DOMO_USERNAME")
        password = creds_json.get("DOMO_PASSWORD")

        access_token = creds_json.get("accessToken") or creds_json.get("ACCESS_TOKEN")

        domo_instance = creds_json.get("instance_url")

        if username and password:
            return cls(
                username=username,
                password=password,
                access_token=access_token,
                domo_instance=domo_instance,
            )

    def __post_init__(self):

        if self.username and self.password:
            print("*** Using Username/Password Auth")
            self._init_session_token_from_username_password_auth()
            return

        if self.access_token:
            print("*** Using Access Token Auth")
            return

        else:
            raise Exception(
                "*** Domo Auth init error: ERROR - valid credential combination not Detected, must pass username and password or session_token"
            )

    def _init_session_token_from_username_password_auth(self):
        url = f"https://{self.domo_instance}/api/content/v2/authentication"

        payload = {
            "method": "password",
            "emailAddress": self.username,
            "password": self.password,
        }

        try:
            response = requests.post(url, json=payload, headers=self.headers, timeout=3)

        except Exception as e:
            print("init_session_token - request error: " + str(e))

        self.session_token = response.json().get("sessionToken")

        if not self.session_token:
            raise Exception(
                f"init_session_token -- unable to retrieve token - {response.text}"
            )

        return self.session_token

    def generate_request_headers(self):
        headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
        }

        if self.session_token:
            headers.update({"x-domo-authentication": self.session_token})
            return headers

        if self.access_token:
            headers.update({"x-domo-developer-token": self.access_token})
            return headers

        raise Exception(
            "generate_request_headers: unable to authenticate request with provided Auth"
        )

    def who_am_i(self, return_raw: bool = False):
        """identify which credentials are being used in this Auth Object (useful for access_token based authentication)"""

        url = f"https://{self.domo_instance}.domo.com/api/content/v2/users/me"

        res = json.loads(
            requests.request(
                method="GET", headers=self.generate_request_headers(), url=url
            ).text
        )

        if return_raw:
            return res

        self.username = res["emailAddress"]

        return res

### sample DomoAuth


In [13]:
auth = DomoAuth(
    domo_instance=os.environ["DOMO_INSTANCE"],
    access_token=os.environ["DOMO_ACCESS_TOKEN"],
)

print(auth.who_am_i(return_raw= False))
auth

*** Using Access Token Auth
{'id': 1893952720, 'invitorUserId': 587894148, 'displayName': 'Jae Wilson1', 'department': 'Business Improvement', 'userName': 'jae@onyxreporting.com', 'emailAddress': 'jae@datacrew.space', 'avatarKey': 'c605f478-0cd2-4451-9fd4-d82090b71e66', 'accepted': True, 'userType': 'USER', 'modified': 1721264727737, 'created': 1588960518, 'active': True, 'pending': False, 'anonymous': False, 'systemUser': False}


DomoAuth(domo_instance='domo-community', username='jae@datacrew.space')

## domo_api_request


In [7]:
# | export


class API_Exception(Exception):

    def __init__(self, res, message: str = None):
        message = message or ""

        base = f" || {str(res.status)} - {res.response.get('message') or res.response.get('statusReason')} || {res.auth.domo_instance}"

        message += base

        super().__init__(message)


@dataclass
class ResponseGetData:
    response: dict
    is_success: bool
    status: int
    auth: DomoAuth = field(repr=False)

    @classmethod
    def from_response(cls, res, auth: DomoAuth):
        return cls(
            response=json.loads(res.text),
            is_success=res.ok,
            status=res.status_code,
            auth=auth,
        )

    @staticmethod
    def _write_stream(res: requests.Response, file_path: str, stream_chunks=8192):

        dmut.upsert_folder(file_path)

        with open(file_path, "wb") as fd:
            for chunk in res.iter_content(stream_chunks):
                fd.write(chunk)

        print("done writing stream")
        return True

    @staticmethod
    def _read_stream(file_name: str):
        with open(file_name, "rb") as f:
            return f.read()

    @classmethod
    def from_stream(cls, res: requests.Response, download_path: str, auth: DomoAuth):

        if not res.ok:
            return cls.from_response(res = res ,auth = auth)


        cls._write_stream(res, download_path)

        return cls(
            response=cls._read_stream(download_path),
            is_success=res.ok,
            status=res.status_code,
            auth=auth,
        )


def domo_api_request(
    auth: DomoAuth,
    endpoint,
    request_type,
    params=None,
    headers=None,
    body=None,
    return_raw: bool = False,
    debug_api: bool = False,
    timeout=3,
):

    url = f"https://{auth.domo_instance}.domo.com{endpoint}"

    headers = headers or {}

    headers = {**auth.generate_request_headers(), **headers}

    if debug_api:

        print("debugging domo_api_request")

        pprint(
            {
                "url": url,
                "headers": headers,
                "request_type": request_type,
                "params": params,
            }
        )

    if request_type.lower() == "post":
        response = requests.post(url, json=body, headers=headers, timeout=timeout)

    elif request_type.lower() == "get":
        response = requests.get(url, headers=headers, params=params, timeout=timeout)

    else:
        raise Exception(
            f'domo_api_request method "{request_type.lower()}" not implemented yet.'
        )

    if return_raw:
        return response

    return ResponseGetData.from_response(res=response, auth=auth)

In [8]:
# | export


def domo_api_stream_request(
    auth: DomoAuth,
    endpoint,
    request_type,
    download_path,
    params=None,
    headers=None,
    debug_api: bool = False,
    timeout=3,
):

    url = f"https://{auth.domo_instance}.domo.com{endpoint}"

    headers = headers or {}

    headers = {**auth.generate_request_headers(), **headers}

    if debug_api:

        print("debugging domo_api_request")

        pprint(
            {
                "url": url,
                "headers": headers,
                "request_type": request_type,
                "params": params,
            }
        )

    res = requests.get(
        url=url, headers=headers, params=params, timeout=timeout, stream=True
    )
    
    if not res.ok:
        return ResponseGetData.from_response(res=res, auth=auth)

    return ResponseGetData.from_stream(
        res=res, download_path=download_path, auth=auth
    )

In [9]:
# | hide
import nbdev

nbdev.nbdev_export("./core.ipynb")